<a href="https://colab.research.google.com/github/itismeganrms/rl-a3/blob/main/reinforce_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb --quiet
!pip install swig --quiet
!pip install gym[all] --quiet
!pip install pygame --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 39.2 MB/s eta 0:00

In [14]:
import gym
from gym.envs import box2d
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [15]:
class Policy_NN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size = 128, gamma = 0.99, learning_rate = 0.001):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

        self.optimizer = optim.Adam(self.parameters(),lr=self.learning_rate)


    def forward(self,x):
      x = self.layer1(x) #forward propagation
      x = self.layer2(x)
      x = self.softmax(x)
      return x

    def act(self,state):
        #With obs, find the maximum Q value and output the corresponding action
        #action_probs = torch.softmax(self.actor(obs), dim=-1)
        #state_value = self.critic(state)

        #obs_tensor = torch.as_tensor(obs, dtype=torch.float32)
        #q_value = self(obs_tensor.unsqueeze(0)) #convert to row vector
        #highest_q_value = torch.argmax(input=q_value)
        #log_value = torch.log(q_value.squeeze(0)[highest_q_value])
        #action = log_value.detach().item() #action corresponding to max Q index
        #return action, log_value
        #print("checkpoint 6")
        action_probs = torch.softmax(self.policy_network(state), dim=-1)
        #print("checkpoint 7")
        #print("action_probs: {}".format(action_probs))
        #state_value = self.critic(state)
        return action_probs

In [16]:
class Reinforce_Update:
    def __init__(self, Policy_NN, rewards, log_value):
        self.network = Policy_NN
        self.rewards = rewards
        self.log_value = log_value

    def update_function(self, gamma, pw):
        discount_rewards = []
        Gt,pw = 0, 0
        for r in reversed(rewards):
            Gt = Gt + gamma**pw * r
            pw = pw + 1
            discount_rewards.append(Gt)

        steps_value = np.arange(rewards.size)

        discount_rewards = torch.tensor(discount_rewards)
        discount_rewards = discount_rewards[::-1].cumsum()[::-1] / gamma**steps_value

        # def discount_rewards(rewards, gamma):
        # t_steps = np.arange(rewards.size)
        # r = rewards * gamma**t_steps
        # r = r[::-1].cumsum()[::-1] / gamma**t_steps
        # return r

        policy_gradient = []
        for log_prob, e_d_r in zip(self.log_value, discount_rewards):
            policy_gradient.append(-log_prob*e_d_r)

        policy_network.optimizer.zero_grad()
        policy_gradient = torch.stack(policy_gradient).sum()
        policy_gradient.backward()
        policy_network.optimizer.step()

In [17]:
def update(policy_network, gamma = 0.99):
    log_probs = []
    rewards = []

In [19]:
import torch
import numpy as np
import gym
import statistics

In [12]:
def main():
    env = gym.make("LunarLander-v2")
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    policy_network = Policy_NN(state_size, action_size)

    print("checkpoint 1")

    episode_array, rewards_array = [] , []
    n_episode = 500
    n_timesteps = 1000

    print("checkpoint 2")

    Reward_list = np.empty(shape=n_episode)

    print("checkpoint 3")

    state = env.reset()
    print("checkpoint 4")

    for episode in range(n_episode):
        log_probs,rewards = [], []
        total_reward = 0
        for step in range(n_timesteps):
            stateT = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
            action_probs = policy_network(stateT)
            action_dist = torch.distributions.Categorical(action_probs)
            action = action_dist.sample()
            #action = action.item()
            #action =
            #actionT = torch.tensor(action)

            log_prob = action_dist.log_prob(action.unsqueeze(0))
            #log_prob = action_dist.log_prob(action)
            #action = torch.argmax(action_probs).item()
            print("Episode:{} and Step: {} and action: {}".format(episode, step,action.item()))
            next_state, reward, done, _ = env.step(action.item())
            total_reward += reward
            rewards.append(reward)
            state = next_state

    discounted_rewards = []
    R = 0
    for reward in rewards:
        R = reward + policy_network.gamma * R
        discounted_rewards.insert(0,R)

    #discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-8)

    # Compute policy loss
    policy_loss = torch.stack(log_probs) * discounted_rewards
    policy_loss = -policy_loss.sum()

    # Update policy
    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
main()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Streaming output truncated to the last 5000 lines.
Episode:11 and Step: 987 and action: 3
Episode:11 and Step: 988 and action: 3
Episode:11 and Step: 989 and action: 3
Episode:11 and Step: 990 and action: 3
Episode:11 and Step: 991 and action: 3
Episode:11 and Step: 992 and action: 3
Episode:11 and Step: 993 and action: 3
Episode:11 and Step: 994 and action: 3
Episode:11 and Step: 995 and action: 3
Episode:11 and Step: 996 and action: 3
Episode:11 and Step: 997 and action: 3
Episode:11 and Step: 998 and action: 3
Episode:11 and Step: 999 and action: 3
Episode:12 and Step: 0 and action: 3
Episode:12 and Step: 1 and action: 3
Episode:12 and Step: 2 and action: 3
Episode:12 and Step: 3 and action: 3
Episode:12 and Step: 4 and action: 3
Episode:12 and Step: 5 and action: 3
Episode:12 and Step: 6 and action: 3
Episode:12 and Step: 7 and action: 3
Episode:12 and Step: 8 and action: 3
Episode:12 and Step: 9 and action: 3
Episode:12 and Step: 10 and action: 3
Episode:12 and Step: 11 and action

KeyboardInterrupt: 

In [ ]:
x = episode_array
y = rewards_array
#plotting the comparison between the two
plt.title("Performance of REINFORCE")
plt.plot(x, y, label = "REINFORCE")
plt.xlabel("Episodes")
plt.ylabel("Rewards")
plt.legend()
plt.show()